In [ ]:
import math
import numpy as np
from numpy import cos, sin, arccos, arctan2, sqrt
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from scipy.optimize import root
from scipy.optimize import minimize

In [ ]:
class ProjetOpt:
  
    def __init__(self):
        #Les configurations initiales des deux joints.
        self.angles_ini = [0, 0]
        #Longueur des deux bras L0 et L1.
        self.bras_longueur = [0.5,0.5]
        self.l0 = self.bras_longueur[0]
        self.l1 = self.bras_longueur[1]
        self.angles = self.joint1 = self.joint2 = [0,0]
        
    def direct(self):
        q0 = self.angles[0] 
        q1 = self.angles[1]
        l0 = self.bras_longueur[0]
        l1 = self.bras_longueur[1]
        self.joint1 = [l0 * cos(q0), l0 * sin(q0)]
        self.joint2 = [l0 * cos(q0) + l1 * cos(q0 + q1), l0 * sin(q0) + l1 * sin(q0 + q1)]
        
        
    def plot(self):   
        plt.cla()    
        plt.xlim(-2, 2)
        plt.ylim(-2, 2)       
        #Les positions des trois joints.
        x = [self.angles_ini[0], self.joint1[0], self.joint2[0]]
        y = [self.angles_ini[1], self.joint1[1], self.joint2[1]]

        plt.plot(x, y, c="red", zorder=1)
      
        plt.scatter(x, y, c="black", zorder=2)
     
        global goal
        plt.scatter(goal[0],goal[1],c='blue',marker='*')
        
        
    def solutionexist(self,goal):
        theta = self.methode1_root(goal)
        def residu(X,goal):
            res = [self.l0 * cos(X[0]) + self.l1 * cos(X[0] + X[1]) - goal[0],
                    self.l0 * sin(X[0]) + self.l1 * sin(X[0] + X[1]) - goal[1]]
            return sqrt(res[0]**2 + res[1]**2)
        
        valeur = residu(theta,goal)
        if valeur < 1e-3:
            vef = True
        else:
            vef = False
        print(f'Le point {goal} est atteignable : ', vef)
    
    

    

    
    """
    ------------------------------------méthode1-------------------------------------------
    """
    def m_root(self,goal):
      #Echantillion N points pour tracer la trajectoire
        N = 10            
        duration_time_seconds = 1
        plt.ion() 
        goal_e = 0
        for i in range(0,N+1):
            goal_e = [goal[0] + (self.l0 + self.l1 - goal[0]) * (1 - i/N), goal[1] * i/N]
            self.angles = self.methode1_root(goal_e)
            #print('solution de theta pour i = ',i, ' est ',self.angles)
            self.direct()
            self.plot()
            clear_output(wait=True)
            dt = duration_time_seconds/N
            plt.pause(dt)


    
    
    def methode1_root(self,goal):
        l0 = self.bras_longueur[0]
        l1 = self.bras_longueur[1]
        param = [goal,self.l0,self.l1]
        def residu(X,param):
            return [l0 * cos(X[0]) + l1 * cos(X[0] + X[1]) - goal[0],
                    l0 * sin(X[0]) + l1 * sin(X[0] + X[1]) - goal[1]]
        solution = root(residu, [1, 1], param, method='hybr')
        theta = solution.x % (2 * math.pi)
        return theta
    
    
    """
    ------------------------------------méthode2-------------------------------------------
    """
    def m_minimize(self,goal,X0):
        #Echantillion N points pour tracer la trajectoire
        N = 10            
        duration_time_seconds = 1
        plt.ion() 
        goal_e = 0
        for i in range(0,N+1):
            goal_e = [goal[0] + (self.l0 + self.l1 - goal[0]) * (1 - i/N), goal[1] * i/N]
            self.angles = self.methode2_minimize(goal_e,X0)
            #print('solution de theta pour i = ',i, ' est ',self.angles)
            self.direct()
            self.plot()
            clear_output(wait=True)
            dt = duration_time_seconds/N
            plt.pause(dt)
         
        


     
     
    def methode2_minimize(self,goal,X0):
        l0 = self.bras_longueur[0]
        l1 = self.bras_longueur[1]
        param = [goal,self.l0,self.l1]
        def residu(X,param):
            res = [l0 * cos(X[0]) + l1 * cos(X[0] + X[1]) - goal[0],
                    l0 * sin(X[0]) + l1 * sin(X[0] + X[1]) - goal[1]]
            return np.linalg.norm(res)
        solution = minimize(residu, X0, param,method='BFGS')
        theta = solution.x % (2 * math.pi)
        return theta
    
    
    """
    ------------------------------------méthode3-------------------------------------------
    """
    
    def m_gradient(self,goal,X0,nmax,alpha):
        def residu(X,goal):
            res = [self.l0 * cos(X[0]) + self.l1 * cos(X[0] + X[1]) - goal[0],
                    self.l0 * sin(X[0]) + self.l1 * sin(X[0] + X[1]) - goal[1]]
            return sqrt(res[0]**2 + res[1]**2)
        #Echantillion N points pour tracer la trajectoirel0 = self.bras_longueur[0]        
        N = 10            
        duration_time_seconds = 1
        plt.ion() 
        goal_e = 0
        calcul =[] 
        for i in range(0,N+1):
            goal_e = [goal[0] + (self.l0 + self.l1 - goal[0]) * (1 - i/N), goal[1] * i/N]
            self.angles, contenu, x_list, y_list = self.methode3_gradient(goal_e,X0,nmax,alpha)
            calcul.append(contenu)
            self.direct()
            self.plot()
            clear_output(wait=True)
            dt = duration_time_seconds/N
            plt.pause(dt)
        
        
        
        
        #Tracer les figures des isovaleurs
        for i in range(0,N+1):
            contenu = calcul[i]
            goal_e = [goal[0] + (self.l0 + self.l1 - goal[0]) * (1 - i/N), goal[1] * i/N]
            self.angles, contenu, x_list, y_list = self.methode3_gradient(goal_e,X0,nmax,alpha)
            
            #Print les informations obtenu a chaque point sur la trajectoire
            print(f"Pour le point N = {i}:")
            for j in range(len(contenu)):    
                print(contenu[j])
            
            
            
            # affichage :           
            plt.figure(i+1)
            xmin, xmax, nx = -7, 7, 50
            ymin, ymax, ny = -7, 7, 50

            # Discrétisation du domaine de tracé
            x1d = np.linspace(xmin,xmax,nx)
            y1d = np.linspace(ymin,ymax,ny)
            x2d,y2d = np.meshgrid(x1d, y1d)
           
            nIso = 10
            plt.contour(x2d,y2d,residu([x2d,y2d],goal),nIso, cmap = 'rainbow')
            for k in range(len(x_list)) :
                plt.plot(x_list[k], y_list[k], 'bs')
            for p in range(len(x_list)-1) :
                plt.plot([x_list[p],x_list[p+1]] , [y_list[p],y_list[p+1]] ,'y' , label = 'segment' )
            plt.title(f"Isovaleurs Pour N = {i}")
            plt.xlabel('Valeurs de x')
            plt.ylabel('Valeurs de y')
            plt.grid()
            

            
    def methode3_gradient(self,goal,X0,nmax,alpha):
        l0 = self.bras_longueur[0]
        l1 = self.bras_longueur[1]
        param = [goal,self.l0,self.l1]
        def residu(X,param):
            res = [l0 * cos(X[0]) + l1 * cos(X[0] + X[1]) - goal[0],
                    l0 * sin(X[0]) + l1 * sin(X[0] + X[1]) - goal[1]]
            return np.linalg.norm(res)
        
        def gradient_J(X,param):
            return np.array([(-2*l0*sin(X[0]) - 2*l1*sin(X[0] + X[1]))*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0]) + (2*l0*cos(X[0]) + 2*l1*cos(X[0] + X[1]))*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1]), 
                             2*l1*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1])*cos(X[0] + X[1]) - 2*l1*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0])*sin(X[0] + X[1])])
    
        
        #Iteration commence
        eps = 1e-2
        Xn = X0
        dX = 1
        dX1 = 1
        n = 0 
        alpha1 = alpha
        #list enregistre de positions
        x_list = [X0[0]]
        y_list = [X0[1]]
        contenu = []
        while (dX > eps) and (n < nmax):
            Xn1 = Xn - alpha * gradient_J(Xn,param)
            dX = np.linalg.norm(Xn1 - Xn)
            Xn = Xn1
            n += 1
            contenu.append(f"Iteration {n}: alpha = {alpha}, dX = {dX}")
            x_list.append(Xn[0])
            y_list.append(Xn[1])
            #La valeur de alpha
            if (dX1 < dX):
                alpha = alpha/2
            else:
                alpha = alpha1    
            dX1 = dX
         
        contenu.append(f"Minimum trouve apres {n} iterations")
        contenu.append(f"Angles du minimum : {Xn}")
        if dX < eps:
            contenu.append('Converge')
        else:
            contenu.append('Diverge')
            
        return Xn, contenu, x_list, y_list
    
    
    """
    ------------------------------------méthode4-------------------------------------------
    """
    
    
    def m_newton(self,goal,X0):
        def residu(X,goal):
            res = [self.l0 * cos(X[0]) + self.l1 * cos(X[0] + X[1]) - goal[0],
                    self.l0 * sin(X[0]) + self.l1 * sin(X[0] + X[1]) - goal[1]]
            return sqrt(res[0]**2 + res[1]**2)
        #Echantillion N points pour tracer la trajectoire
        N = 10            
        duration_time_seconds = 1
        plt.ion() 
        goal_e = 0
        for i in range(0,N+1):
            goal_e = [goal[0] + (self.l0 + self.l1 - goal[0]) * (1 - i/N), goal[1] * i/N]
            self.angles,x_list,y_list = self.methode4_newton(goal_e,X0)
            self.direct()
            self.plot()
            clear_output(wait=True)
            dt = duration_time_seconds/N
            plt.pause(dt)
         
        #Tracer les figures des isovaleurs
        for i in range(0,N+1):
            goal_e = [goal[0] + (self.l0 + self.l1 - goal[0]) * (1 - i/N), goal[1] * i/N]
            self.angles,x_list,y_list = self.methode4_newton(goal_e,X0)
            plt.figure(i+1)
            xmin, xmax, nx = -10, 10, 50
            ymin, ymax, ny = -10, 10, 50
    
            # Discrétisation du domaine de tracé
            x1d = np.linspace(xmin,xmax,nx)
            y1d = np.linspace(ymin,ymax,ny)
            x2d,y2d = np.meshgrid(x1d, y1d)
           
            nIso = 10
            plt.contour(x2d,y2d,residu([x2d,y2d],goal),nIso, cmap = 'rainbow')
            plt.title(f"Isovaleurs Pour N = {i}")
            plt.xlabel('Valeurs de x')
            plt.ylabel('Valeurs de y')
            plt.grid()    
            for k in range(len(x_list)) :
                plt.plot(x_list[k], y_list[k], 'bs')
            for p in range(len(x_list)-1) :
                plt.plot([x_list[p],x_list[p+1]] , [y_list[p],y_list[p+1]] ,'y' , label = 'segment' )
    
    
    
    def methode4_newton(self,goal,X0):
        l0 = self.bras_longueur[0]
        l1 = self.bras_longueur[1]
        param = [goal,self.l0,self.l1]
        
        
        #calcule de gradient de J
        def gradient_J(X,param):
            return np.array([(-2*l0*sin(X[0]) - 2*l1*sin(X[0] + X[1]))*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0]) + (2*l0*cos(X[0]) + 2*l1*cos(X[0] + X[1]))*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1]), 
                             2*l1*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1])*cos(X[0] + X[1]) - 2*l1*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0])*sin(X[0] + X[1])])
        #clacule de Hessien de J
        def hessien_J(X,paran):
               return np.array([[(-2*l0*sin(X[0]) - 2*l1*sin(X[0] + X[1]))*(-l0*sin(X[0]) - l0*sin(X[0] + X[1])) + (-2*l0*sin(X[0]) - 2*l1*sin(X[0] + X[1]))*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1]) + (-2*l0*cos(X[0]) - 2*l1*cos(X[0] + X[1]))*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0]) + (l0*cos(X[0]) + l1*cos(X[0] + X[1]))*(2*l0*cos(X[0]) + 2*l1*cos(X[0] + X[1])), -l1*(-2*l0*sin(X[0]) - 2*l0*sin(X[0] + X[1]))*sin(X[0] + X[1]) + l1*(2*l0*cos(X[0]) + 2*l0*cos(X[0]+ X[1]))*cos(X[0] + X[1]) - 2*l1*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1])*sin(X[0] + X[1]) - 2*l1*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0])*cos(X[0] + X[1])], 
                             [-2*l1*(-l0*sin(X[0]) - l1*sin(X[0] + X[1]))*sin(X[0] + X[1]) + 2*l1*(l0*cos(X[0]) + l1*cos(X[0] + X[1]))*cos(X[0] + X[1]) - 2*l1*(l0*sin(X[0]) + l1*sin(X[0] + X[1]) - goal[1])*sin(X[0]+ X[1]) - 2*l0*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0])*cos(X[0] + X[1]), 2*l1**2*sin(X[0] + X[1])**2 + 2*l1**2*cos(X[0] + X[1])**2 - 2*l1*(l0*sin(X[0]) + l0*sin(X[0] + X[1]) - goal[1])*sin(X[0] + X[1]) - 2*l1*(l0*cos(X[0]) + l1*cos(X[0] + X[1]) - goal[0])*cos(X[0] + X[1])]])
        
        #Iteration commence
        x_n = X0
        eps = 1e-10
        n_max = 50000
        dx = 1
        n = 0
        
        #list enregistre de angles: x_list pour theta0 et y_list pour theta1
        x_list = [X0[0]]
        y_list = [X0[1]]
        
        while dx > eps and n < n_max:
            J = gradient_J(x_n,param)
            H = hessien_J(x_n,param)
            delta_x = -np.dot(np.linalg.inv(H),J)
            x_n = x_n + delta_x
            dx = np.linalg.norm(delta_x)
            n += 1
            x_list.append(x_n[0])
            y_list.append(x_n[1])
    
        return x_n,x_list,y_list

In [ ]:
"""
-------------------------test--------------------------- 
"""
#Les points desires enregestres et a choisir, vous pouvez juste changer le chiffre dans la list goals:
#ex.:  goal = goals[0], goal = goals[1]. 
#Ne changez et remplacez pas le nom de point desire "goal":
#ex.:  a = goals[0], ca ne marche pas
goals = [[0.55,0.4],[-0.5,0.4]]

#Fonctionner la classe
result = ProjetOpt()
for goal in goals:
    result.solutionexist(goal)

# Methode1_root test

***Test goal1***

In [ ]:
goal = goals[0] 
result.m_root(goal)

***Test goal2***

In [ ]:
goal = goals[1] 
result.m_root(goal)

# Methode2_minimize test

***Test goal1***

In [ ]:
goal = goals[0] 
X0 = [1,1]
result.m_minimize(goal,X0)

***Test goal2***

In [ ]:
goal = goals[1] 
X0 = [1,1]
result.m_minimize(goal,X0)

# Methode3_gradient test

***Test goal1***

In [ ]:
goal = goals[0] 
X0 = [1,1]
nmax = 5000
alpha = 1
result.m_gradient(goal,X0,nmax,alpha)

***Test goal2***

In [ ]:
goal = goals[1] 
X0 = [1,1]
nmax = 5000
alpha = 1
result.m_gradient(goal,X0,nmax,alpha)

# Methode4_newton test

***Test goal1***

In [ ]:
goal = goals[0] 
X0 = [1,1]
result.m_newton(goal,X0)

***Test goal2***

In [ ]:
goal = goals[1] 
X0 = [1,1]
result.m_newton(goal,X0)

## Importance du pas de la méthode du gradient 

+ Lorsque la valeur de pas est grande, nous n'obtiendrons pas la solution optimale, car nous pouvons observer que le point d'itération oscillera autour de la solution optimale ; et lorsque la valeur de pas est petite, nous obtiendrons certainement la solution optimale, mais il augmentera considérablement la quantité de calcul. Il est donc nécessaire de modifier manuellement la valeur de pas pour obtenir un résultat de calcul approprié.

## Importance du point de départ de la méthode de Newton

+ Notre angle de point initial est (0, 0)(La position initiale d'effecteur est (L1+L2,0), dans cet exmple est (1,0)) et notre point cible est (x,y) = (0,55, 0,4).

+ Pour obtenir des résultats plus précis, nous choisissons X0(1, 1) comme valeur d'angle initiale de l'itération, mais en exécutant le programme, nous observons qu'aux cinq premiers points de la trajectoire, nous obtenons une solution inappropriée. En examinant les tracés de l'isovaleur pour chaque point passé sur la trajectoire, nous constatons qu'il existe de nombreux extrêmes locaux dans les tracés de l'isovaleur, et en examinant les cinq premiers tracés, nous constatons que les solutions que nous obtenons est approché à d'autres extrêmes que ceux que nous souhaitons. Nous considérons donc qu'il n'est pas approprié d'utiliser la méthode de Newton lorsque la distance entre le point de départ et le point cible est trop grande. Nous devrions donc choisir un point proche du point de valeur extrême désiré comme point de départ de l'itération.

## Existence de solutions non uniques 

+ Pour le point où il y a une solution, on peut juger en observant l'image obtenue. Lorsque le point effecteur terminal est sur le cercle avec l1+l2 comme r, il n'y a qu'une seule solution. En se déplaçant dans ce cercle, il y aura deux solutions.